[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Tiffany Moran, 2020 ([t.c.moran@se21.qmul.ac.uk](mailto:t.c.moran@se21.qmul.ac.uk)) All rights reserved.

# NHS COVID-19 Dashboard

In this COVID-19 Dashboard you will see an information that takes data from [Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE) and displays it in an interactive graph.

The data that I have chosen to focus in on is vaccination rates across England over time, broken down by first dose, second dose, and boosters/third doses. This is because I was interested in seeing if certain events and dates correlated with any significant increase in vaccination rates. For example, with the recent news of the Omicron variant, you would be able to specify the week leading up to the news and the weeks after to see if an uptick in vaccinations occurred. 

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [16]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [17]:
# Load JSON files and store the raw data
f=open('vaccinationtracker.json')
vaccinationdata=json.load(f)

In [4]:
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=rawdata['data']
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['dose1', 'dose2', 'dose3'])
    
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['dose1', 'dose2', 'dose3']:
            if pd.isna(df.loc[date, column]):
                value=float(entry[column]) if entry [column] != None else 0.0
                df.loc[date, column]=value
    
    df.fillna(0.0, inplace=True)
    return df


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(vaccinationdata) # df is the dataframe for plotting



In [5]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=overview'
    ]

    structure = {
        'date':'date',
        'dose1':'cumVaccinationFirstDoseUptakeByPublishDatePercentage',
        'dose2':'cumVaccinationSecondDoseUptakeByPublishDatePercentage',
        'dose3':'cumVaccinationThirdInjectionUptakeByPublishDatePercentage'
    }

    api = Cov19API(filters=filters, structure=structure)

    refreshdata=api.get_json()
    return refreshdata # return data read from the API



In [6]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data', # you may want to change this...
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click here to pull the latest COVID-19 data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets


# run all cells before clicking on this button


## Vaccination Rate Graph and Analysis

This graph depicts dates for 2021 on the x-axis and the total vaccinated percentage of the English population on they y-axis. 

<b>Press to Refresh:</b>
The graph presented might not have the most up-to-date data from PHE. You can click on the refresh button below to update the graph with the most recent dates and vaccination rates.

<b>Vaccine Doses:</b>
You are able to filter first doses, second doses, and third doses (defined as a combination of boosters for the general population and third doses for severely at risk individuals). These can be filtered one by one or multiple categories can be viewed simultaneously by CTRL+clicking.

<b>Dates:</b>
The dates included are all of the possible dates available, starting from January 10th 2021 to the most recent dates. The range of dates can be altered on the slider, which will subsequently update the graph.

<b>Interesting Dates to Filter For:</b>
- February 28th 2021 - 60+ Eligible for Vaccine
- March 17th 2021 - 100th Day of Vaccine Program and 50+ Eligible for Vaccine
- April 30th 2021 - 40+ Eligible for Vaccine
- May 26th 2021 - 30+ Eligible for Vaccine
- June 15th 2021 - CDC declares Delta Variant a concern
- June 17th 2021 - All Adults Eligible for Vaccine
- September 16th 2021 - NHS begins booster campaign
- November 26th 2021 - WHO announces Omicron variant a concern



In [ ]:
series=wdg.SelectMultiple(
    options=['dose1', 'dose2', 'dose3'],
    value=['dose1', 'dose2', 'dose3'],
    rows=3,
    description='Vaccine Doses:',
    disabled=False,
    layout={'width': '400px'}
)

dates = [(i.strftime('%b %d'), i) for i in df.index]
slide=wdg.SelectionRangeSlider(
    options=dates,
    index=(0, len(dates)-1),
    description='Dates (2021):',
    disabled=False,
    continuous_update=False,
    tooltips="Select date range for chart",
    layout={'width': '520px'}
)

controls=wdg.VBox([series, slide])

def vaccinationtracker_graph(gcols, gslide):
    ncols=len(gcols)
    gslide=parse_date(slide.value)
    plt.show()
    if ncols>0:
        df[list(gcols)].loc[slide.value[0]:slide.value[1]].plot(colormap='Set3',
         title='UK Vaccination Rate Over Time', xlabel='Dates', ylabel='% Percent of Vaccinated Population')
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    if slide.value[0]==slide.value[1]:
        print("Select a bigger range of dates for graph")

def refresh_graph():
    dates = [(i.strftime('%b %d'), i) for i in df.index]
    slide.options=dates
    slide.index=(0, len(dates)-1)
  
graph=wdg.interactive_output(vaccinationtracker_graph, {'gcols': series, 'gslide': slide})

display(graph, controls, apibutton)




Output()

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click here to pull the lates…

**Author and Copyright Notice**  
*(C) Tiffany Moran, 2020 all rights reserved.* <br>
*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*